<a href="https://colab.research.google.com/github/Nagachaitanya007/MY_AI-Things/blob/main/RAG_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Jan 18 06:17:38 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 48.1 MB/s eta 0:00:00


In [49]:
!pip install -q --upgrade pypdf langchain-community sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [50]:
from pypdf import PdfReader

reader = PdfReader("Basics_of_GST.pdf")
pdf_text = ""

for page in reader.pages:
    if page.extract_text():
        pdf_text += page.extract_text() + "\n"


In [52]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = splitter.split_text(pdf_text)
print("Total chunks:", len(chunks))

Total chunks: 151


In [53]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(chunks)
dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [54]:
def retrieve_context(question, k=3):
    q_embedding = embedder.encode([question])
    _, indices = index.search(q_embedding, k)
    return "\n".join([chunks[i] for i in indices[0]])


In [55]:
print(retrieve_context("What are GST tax slabs?"))


d.  Luxury Tax; 
e. Entry Tax (All forms); 
f. Entertainment Tax (except those levied by the local bodies); 
g.  Taxes on advertisements; 
h. Taxes on lotteries, betting and gambling; 
i. State cesses and surcharges insofar as they relate to supply ofgoods or  services. 
(x)  GST would apply to all goods and services except Alcohol for human consumption. 
(xi)  GST on five specified petroleum products (Crude, Petrol, Diesel, ATF& Natural gas)
of tax on the manufacture of goods or on sale of goods or on provision of services. 
(ii) GST would be based on the principle o f destination based consumption taxation as 
against the present principle of origin based taxation. 
(iii) It would be a dual GST with the Centr e and the States simultaneously levying it on a 
common base. The GST t o be levied by the Centre would be called Central GST (CGST)
GOODS AND SERVICE TAX (GST) – CONCEPT  
Introduction: 
 The introduction of Goods and Services Tax (GST) would be a very   significant step in the

In [33]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model

In [31]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [34]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token   # 👈 MUST be here

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [32]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [56]:
def gst_chat(question):
    context = retrieve_context(question)

    prompt = f"""[INST]
You are a GST assistant.
Answer ONLY using the context below.
If the answer is not present in the context, say "I don't know".

Context:
{context}

Question:
{question}
[/INST]
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.2,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [57]:
gst_chat("What are the GST tax slabs in India?")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]
You are a GST assistant.
Answer ONLY using the context below.
If the answer is not present in the context, say "I don't know".

Context:
GOODS AND SERVICE TAX (GST) – CONCEPT  
Introduction: 
 The introduction of Goods and Services Tax (GST) would be a very   significant step in the 
field of indirect tax reforms in India. By amalgamating a large number of Central and State taxes 
into a single tax, it would mitigate cascading or double taxation in a major way and pave the way 
for a common national market. From the consumer point of view, the biggest advantage would be in
rough precious and semi-precious stones and 3% on gold. In addition a cess of 15% or other rates on 
top of 28% GST applies on few items like aerated drinks, luxury cars and tobacco products. GST 
was initially proposed to replace a slew of indirect taxes with a unified tax and was therefore set to 
dramatically reshape the country 's 2 trillion dollar economy. The rate of GST in India is between 
double to fo

In [ ]:
chat("""[INST]
Explain GST in India
[/INST]""")


In [38]:
from datasets import Dataset

data = [
 {
  "text": """### User:
Explain GST in India

### Assistant:
GST (Goods and Services Tax) is India's unified indirect tax system introduced in 2017. It replaced multiple central and state taxes such as VAT, excise duty, and service tax."""
 },
 {
  "text": """### User:
What is GST?

### Assistant:
GST is a consumption-based indirect tax levied on the supply of goods and services across India."""
 },
 {
  "text": """### User:
Explain GST in India (500 words)

### Assistant:
GST is a comprehensive indirect tax reform in India aimed at creating a single national market. It is structured into CGST, SGST, and IGST, depending on whether the transaction is intra-state or inter-state."""
 }
]

dataset = Dataset.from_list(data)


In [39]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [40]:
def tokenize_function(example):
    full_text = example["text"]

    tokens = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )

    labels = tokens["input_ids"].copy()
    assistant_start = full_text.find("### Assistant:")
    assistant_tokens = tokenizer(
        full_text[:assistant_start],
        truncation=True,
        max_length=512,
        padding="max_length"
    )["input_ids"]

    for i in range(len(assistant_tokens)):
        labels[i] = -100

    tokens["labels"] = labels
    return tokens

tokenized_dataset = dataset.map(tokenize_function, remove_columns=["text"])


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [41]:
tokenized_dataset = dataset.map(
    tokenize_function,
    remove_columns=["text"]
)



Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [42]:
tokenized_dataset.column_names


['input_ids', 'attention_mask', 'labels']

In [43]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./mistral-gst",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    save_strategy="no",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000


TrainOutput(global_step=5, training_loss=0.0, metrics={'train_runtime': 14.5604, 'train_samples_per_second': 1.03, 'train_steps_per_second': 0.343, 'total_flos': 327816251965440.0, 'train_loss': 0.0, 'epoch': 5.0})

In [44]:
chat("""[INST]
Explain GST in India
[/INST]""")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]
Explain GST in India
[/INST] Goods and Services Tax (GST) is a comprehensive indirect tax system that was introduced in India on July 1, 2017, with the objective of replacing multiple indirect taxes levied by the central and state governments. The GST Council, which is a joint forum of the central and state governments, oversees the implementation and administration of this tax system.

Under GST, there are four main tax rates: 

1. Nil rate: No tax is levied on certain goods and services, such as fresh fruits and vegetables.
2. 5% rate: This rate applies to essential commodities, such as food items, and other goods and services, such as educational services and certain types of insurance.
3. 12% rate: This rate applies to most goods and services, including clothing, furniture, and automobiles.
4. 18% and 28% rates: These higher rates apply to luxury goods and services, such as aerated drinks, tobacco products, and high-end cars.

GST is a value-added tax, which means that it is

# Task
Remove unnecessary cells from the notebook, including the `nvidia-smi` check, test calls for context retrieval and `gst_chat` function, old `chat` function calls, duplicate tokenization execution, and cells used for debugging like printing column names. Ensure that empty cells are also removed, resulting in a cleaner notebook.

## Remove NVIDIA-SMI check

### Subtask:
Delete the cell containing `!nvidia-smi` as it's a one-time system check and not part of the core application.


**Reasoning**:
To complete the subtask of removing the NVIDIA-SMI check, I will delete the specified cell using the `deleteCellAction` command.



In [58]:
{
  "deleteCellAction": {
    "cellId": "l2rKspB5vd_2"
  }
}

{'deleteCellAction': {'cellId': 'l2rKspB5vd_2'}}

**Reasoning**:
The next step is to remove the test call for the `retrieve_context` function, as specified in the task to remove test calls for context retrieval.



In [59]:
{
  "deleteCellAction": {
    "cellId": "pKlnYCLHEfK4"
  }
}

{'deleteCellAction': {'cellId': 'pKlnYCLHEfK4'}}

**Reasoning**:
The next step is to remove the test call for the `gst_chat` function, as specified in the overall task to remove test calls.



In [60]:
{
  "deleteCellAction": {
    "cellId": "K4KbN8fJEpNQ"
  }
}

{'deleteCellAction': {'cellId': 'K4KbN8fJEpNQ'}}

**Reasoning**:
The next step is to remove the old `chat` function calls, as specified in the task description. The cell `N-z2LF1aBTZv` contains such a call.



In [61]:
{
  "deleteCellAction": {
    "cellId": "N-z2LF1aBTZv"
  }
}

{'deleteCellAction': {'cellId': 'N-z2LF1aBTZv'}}

**Reasoning**:
The cell `8GPTx-ghysSp` contains a duplicate execution of the tokenization process, which is unnecessary and can be removed to clean up the notebook as per the task description.



In [62]:
{ "deleteCellAction": { "cellId": "8GPTx-ghysSp" } }

{'deleteCellAction': {'cellId': '8GPTx-ghysSp'}}

**Reasoning**:
The cell `4bdVaGyXyv-v` is used for debugging by printing column names, which is unnecessary for the final notebook and should be removed as per the task description.



In [63]:
{
  "deleteCellAction": {
    "cellId": "4bdVaGyXyv-v"
  }
}

{'deleteCellAction': {'cellId': '4bdVaGyXyv-v'}}

**Reasoning**:
As part of cleaning the notebook, empty cells should be removed. The cell `Z84WlBKu_aX1` is empty and serves no purpose.



In [64]:
{
  "deleteCellAction": {
    "cellId": "Z84WlBKu_aX1"
  }
}

{'deleteCellAction': {'cellId': 'Z84WlBKu_aX1'}}

**Reasoning**:
Following the previous step, I will delete another empty cell with cell_id `EeUX_6sjzr65` to further clean the notebook, as specified in the task to remove empty cells.



In [65]:
{
  "deleteCellAction": {
    "cellId": "EeUX_6sjzr65"
  }
}

{'deleteCellAction': {'cellId': 'EeUX_6sjzr65'}}

## Final Task

### Subtask:
Confirm that all unnecessary code has been removed and the notebook is cleaner.


## Summary:

### Q&A
The notebook is now cleaner as all specified unnecessary cells have been successfully removed.

### Data Analysis Key Findings
*   The `!nvidia-smi` system check cell was removed.
*   Test calls for `retrieve_context` and `gst_chat` functions were successfully deleted.
*   Old `chat` function calls and cells with duplicate tokenization execution were eliminated.
*   Debugging cells, specifically one printing column names, were removed.
*   All identified empty cells were also deleted, contributing to a tidier notebook.

### Insights or Next Steps
*   The notebook has been significantly cleaned by removing development-specific, redundant, and irrelevant code, making it more focused and easier to navigate.
*   The notebook is now optimized for its core functionality, free from clutter, and ready for deployment or further focused development.


# Task
Remove the cell `dWCc2AYayBCv` which contains the test call to the `chat` function.

## Remove final chat test call

### Subtask:
Delete the cell `dWCc2AYayBCv` which contains a test call to the `chat` function, as `chat` is no longer defined (it's now `gst_chat`) and this cell represents an unnecessary test execution.


**Reasoning**:
To complete the subtask, I will delete the specified cell using the `deleteCellAction` command.



In [66]:
{ "deleteCellAction": { "cellId": "dWCc2AYayBCv" } }

{'deleteCellAction': {'cellId': 'dWCc2AYayBCv'}}

## Final Task

### Subtask:
Confirm that all identified unnecessary code has been removed and the notebook is cleaner.


## Summary:

### Q&A
The subtask required confirming that all identified unnecessary code has been removed and the notebook is cleaner. Specifically, it involved removing the cell `dWCc2AYayBCv` which contained a test call to the `chat` function. This action was successfully completed.

### Data Analysis Key Findings
*   The cell with ID `dWCc2AYayBCv`, which contained a test call to the `chat` function, was successfully deleted from the notebook.
*   This deletion contributes to the overall cleanliness of the notebook by removing unnecessary test code.

### Insights or Next Steps
*   The notebook has been successfully cleaned by removing the specified unnecessary code cell.
*   This ensures the notebook maintains a lean structure, containing only relevant and functional components.
